#**Исследование датасета 2**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/RoadWorksAnalyzer/data/

/content/drive/My Drive/Colab Notebooks/RoadWorksAnalyzer/data


In [ ]:
# Импортируем нужные библиотеки
import re

import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import LineString

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Moscow-alter-2024-11-13.csv', sep=';')
df.head()

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,Contractor,global_id,geoData,geodata_center
0,Место проведения,Год проведения работ,Объект расположен в пределах границ города Москвы,Административный округ,Район,Плановая дата начала выполнения работ,Плановая дата окончания работ,Фактическая дата начала выполнения работ,Фактическая дата окончания работ,Тип работ,Статус работ,Основание проведения работ,Заказчик,Подрядчик,global_id,geoData,geodata_center
1,2-й Звенигородский переулок,2020,да,Центральный административный округ,Пресненский район,14.08.2020,08.10.2020,14.08.2020,08.10.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138031,"{coordinates=[[[37.555009201, 55.764145825], [...","{coordinates=[37.555888069, 55.765439715], typ..."
2,2-я Звенигородская улица,2020,да,Центральный административный округ,Пресненский район,28.08.2020,13.09.2020,28.08.2020,13.09.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138034,"{coordinates=[[[37.555076618, 55.75643665], [3...","{coordinates=[37.554847163, 55.759759371], typ..."
3,2-я Черногрязская улица,2020,да,Центральный административный округ,Пресненский район,14.08.2020,02.10.2020,14.08.2020,02.10.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138035,"{coordinates=[[[37.547976298, 55.76482853], [3...","{coordinates=[37.547257466, 55.762548058], typ..."
4,Бахрушина ул.,2020,да,Центральный административный округ,район Замоскворечье,21.03.2020,03.06.2020,21.03.2020,03.06.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138040,"{coordinates=[[[37.637720128, 55.731480493], [...","{coordinates=[37.635806609, 55.734396363], typ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3591 entries, 0 to 3590
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   WorksPlace           3591 non-null   object
 1   WorkYear             3591 non-null   object
 2   OnTerritoryOfMoscow  3591 non-null   object
 3   AdmArea              3568 non-null   object
 4   District             3568 non-null   object
 5   WorksBeginDate       3591 non-null   object
 6   PlannedEndDate       3591 non-null   object
 7   ActualBeginDate      3520 non-null   object
 8   ActualEndDate        3345 non-null   object
 9   WorksType            3591 non-null   object
 10  WorksStatus          3591 non-null   object
 11  WorkReason           3591 non-null   object
 12  Customer             2459 non-null   object
 13  Contractor           24 non-null     object
 14  global_id            3591 non-null   object
 15  geoData              3591 non-null   object
 16  geodat

##**Первоначальная очистка данных**

Проверка признаков на Nan

In [ ]:
df.isna().sum()

,0
WorksPlace,0
WorkYear,0
OnTerritoryOfMoscow,0
AdmArea,23
District,23
WorksBeginDate,0
PlannedEndDate,0
ActualBeginDate,71
ActualEndDate,246
WorksType,0


Первая строка датасета выступает в качестве описания датасета. Уберем ее из данных

In [ ]:
column_descriptions = df.iloc[0].tolist()
column_descriptions

['Место проведения',
 'Год проведения работ',
 'Объект расположен в пределах границ города Москвы',
 'Административный округ',
 'Район',
 'Плановая дата начала выполнения работ',
 'Плановая дата окончания работ',
 'Фактическая дата начала выполнения работ',
 'Фактическая дата окончания работ',
 'Тип работ',
 'Статус работ',
 'Основание проведения работ',
 'Заказчик',
 'Подрядчик',
 'global_id',
 'geoData',
 'geodata_center']

In [ ]:
df = df.drop(0)
df.head()

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,Contractor,global_id,geoData,geodata_center
1,2-й Звенигородский переулок,2020,да,Центральный административный округ,Пресненский район,14.08.2020,08.10.2020,14.08.2020,08.10.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138031,"{coordinates=[[[37.555009201, 55.764145825], [...","{coordinates=[37.555888069, 55.765439715], typ..."
2,2-я Звенигородская улица,2020,да,Центральный административный округ,Пресненский район,28.08.2020,13.09.2020,28.08.2020,13.09.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138034,"{coordinates=[[[37.555076618, 55.75643665], [3...","{coordinates=[37.554847163, 55.759759371], typ..."
3,2-я Черногрязская улица,2020,да,Центральный административный округ,Пресненский район,14.08.2020,02.10.2020,14.08.2020,02.10.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138035,"{coordinates=[[[37.547976298, 55.76482853], [3...","{coordinates=[37.547257466, 55.762548058], typ..."
4,Бахрушина ул.,2020,да,Центральный административный округ,район Замоскворечье,21.03.2020,03.06.2020,21.03.2020,03.06.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138040,"{coordinates=[[[37.637720128, 55.731480493], [...","{coordinates=[37.635806609, 55.734396363], typ..."
5,Библиотечная ул. (от ул. Б.Андроньевская до ул...,2020,да,Центральный административный округ,Таганский район,08.03.2020,20.03.2020,08.03.2020,20.03.2020,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,NaN,1031138042,"{coordinates=[[[37.668564862, 55.744265971], [...","{coordinates=[37.66860947, 55.744302349], type..."


In [ ]:
descriptions_df = pd.DataFrame(column_descriptions, columns=['Descriptions'], index=df.columns)

print("Column Descriptions:")
print(descriptions_df)

Column Descriptions:
                                                          Descriptions
WorksPlace                                            Место проведения
WorkYear                                          Год проведения работ
OnTerritoryOfMoscow  Объект расположен в пределах границ города Москвы
AdmArea                                         Административный округ
District                                                         Район
WorksBeginDate                   Плановая дата начала выполнения работ
PlannedEndDate                           Плановая дата окончания работ
ActualBeginDate               Фактическая дата начала выполнения работ
ActualEndDate                         Фактическая дата окончания работ
WorksType                                                    Тип работ
WorksStatus                                               Статус работ
WorkReason                                  Основание проведения работ
Customer                                                

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 1 to 3590
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   WorksPlace           3590 non-null   object
 1   WorkYear             3590 non-null   object
 2   OnTerritoryOfMoscow  3590 non-null   object
 3   AdmArea              3567 non-null   object
 4   District             3567 non-null   object
 5   WorksBeginDate       3590 non-null   object
 6   PlannedEndDate       3590 non-null   object
 7   ActualBeginDate      3519 non-null   object
 8   ActualEndDate        3344 non-null   object
 9   WorksType            3590 non-null   object
 10  WorksStatus          3590 non-null   object
 11  WorkReason           3590 non-null   object
 12  Customer             2458 non-null   object
 13  Contractor           23 non-null     object
 14  global_id            3590 non-null   object
 15  geoData              3590 non-null   object
 16  geodat

In [ ]:
df.describe(include = "all")

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,Contractor,global_id,geoData,geodata_center
count,3590,3590,3590,3567,3567,3590,3590,3519,3344,3590,3590,3590,2458,23,3590,3590,3590
unique,3514,5,2,36,464,551,464,783,734,1,3,1,1,1,3590,3583,3582
top,"Пятницкое шоссе, 54 к.2 (дорога от Пятницкого ...",2024,да,Центральный административный округ,Таганский район,15.04.2024,31.10.2024,24.04.2024,30.09.2024,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,ГБУ АВД,1031138031,"{coordinates=[[[37.751362324, 55.788066338], [...","{coordinates=[37.667591572, 55.749127688], typ..."
freq,2,1018,3567,591,77,1018,773,58,146,3590,3344,3590,2458,23,1,2,2


In [ ]:
df = df.drop(['Contractor'], axis=1)

In [ ]:
df[df.AdmArea.isna()]['OnTerritoryOfMoscow'].value_counts()

,count
OnTerritoryOfMoscow,
нет,23


Все адреса, у которых отсутствует значение поля AdmArea или District, находятся вне территории Москвы. Заменим Nan на 'Московская область'

In [ ]:
df['AdmArea'] = df['AdmArea'].fillna('Московская область')

In [ ]:
df['District'] = df['District'].fillna('Московская область')

In [ ]:
df.Customer.value_counts()

,count
Customer,
ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,2458


In [ ]:
df['Customer'] = df['Customer'].fillna('Unknown')

##**Изменение типов данных**

In [ ]:
df['WorksBeginDate'] = pd.to_datetime(df['WorksBeginDate'], format='%d.%m.%Y',  errors='coerce')
df['PlannedEndDate'] = pd.to_datetime(df['PlannedEndDate'], format='%d.%m.%Y',  errors='coerce')
df['ActualBeginDate'] = pd.to_datetime(df['ActualBeginDate'], format='%d.%m.%Y',  errors='coerce')
df['ActualEndDate'] = pd.to_datetime(df['ActualEndDate'], format='%d.%m.%Y',  errors='coerce')

In [ ]:
df.sample(5)

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,global_id,geoData,geodata_center
3024,Печорская улица,2024,да,Северо-Восточный административный округ,Бабушкинский район,2024-04-15,2024-11-30,2024-05-13,NaT,текущий ремонт асфальтобетонного покрытия,идут,Титульный список объектов улично-дорожной сети...,Unknown,2644262359,"{coordinates=[[[37.655961514, 55.86548686], [3...","{coordinates=[37.670037746, 55.863439886], typ..."
1618,2-й Пятигорский проезд,2022,да,Юго-Восточный административный округ,Рязанский район,2022-04-15,2022-10-31,2022-09-07,2022-09-16,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1266596181,"{coordinates=[[[37.766232491, 55.725626799], [...","{coordinates=[37.766232491, 55.725626799], typ..."
2092,Ляпидевского улица на всем протяжении,2023,да,Северный административный округ,"[Головинский район, район Ховрино]",2023-07-15,2023-08-05,2023-06-28,2023-08-06,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,2579865604,"{coordinates=[[[37.492260933, 55.859598507], [...","{coordinates=[37.492260933, 55.859598507], typ..."
94,Маршала Жукова проспект от ул.Генерала Глаголе...,2020,да,"[Западный административный округ, Северо-Запад...","[район Крылатское, район Хорошёво-Мнёвники]",2020-03-26,2020-05-09,2020-03-26,2020-05-09,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1032432806,"{coordinates=[[[37.432182613, 55.772963651], [...","{coordinates=[37.445990625, 55.777426489], typ..."
1237,1-й Дорожный проезд,2021,да,Новомосковский административный округ,поселение Кокошкино,2021-09-21,2021-10-10,2021-09-15,2021-11-01,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1132637420,"{coordinates=[[[37.180110812, 55.603334238], [...","{coordinates=[37.180110812, 55.603334238], typ..."


In [ ]:
df['WorksStatus'].value_counts()

,count
WorksStatus,
закончены,3344
идут,175
не начаты,71


In [ ]:
df[df['WorksStatus'] == 'идут'].head()

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,global_id,geoData,geodata_center
2527,Правая Дворцовая аллея,2023,да,Северный административный округ,район Аэропорт,2023-04-15,2023-10-30,2023-08-22,NaT,текущий ремонт асфальтобетонного покрытия,идут,Титульный список объектов улично-дорожной сети...,Unknown,2609703904,"{coordinates=[[[37.553792149, 55.793970051], [...","{coordinates=[37.552925795, 55.792730572], typ..."
2572,Филаретовская улица (пр. 5372),2024,да,Зеленоградский административный округ,район Силино,2024-04-15,2024-11-30,2024-06-10,NaT,текущий ремонт асфальтобетонного покрытия,идут,Титульный список объектов улично-дорожной сети...,Unknown,2643801195,"{coordinates=[[[37.168405652, 55.990852048], [...","{coordinates=[37.177122831, 55.99266428], type..."
2574,Абельмановская ул. д.17 (кинотеатр победа) и д...,2024,да,Центральный административный округ,Таганский район,2024-04-15,2024-11-30,2024-04-27,NaT,текущий ремонт асфальтобетонного покрытия,идут,Титульный список объектов улично-дорожной сети...,Unknown,2643801239,"{coordinates=[[[37.670080662, 55.734570976], [...","{coordinates=[37.670552731, 55.734432041], typ..."
2576,Проезд 4921,2024,да,Зеленоградский административный округ,район Савёлки,2024-04-15,2024-11-30,2024-04-26,NaT,текущий ремонт асфальтобетонного покрытия,идут,Титульный список объектов улично-дорожной сети...,Unknown,2643801260,"{coordinates=[[[37.254520655, 55.980835984], [...","{coordinates=[37.256510854, 55.983391448], typ..."
2579,Абельмановской заставы пл.,2024,да,Центральный административный округ,Таганский район,2024-04-15,2024-11-30,2024-08-20,NaT,текущий ремонт асфальтобетонного покрытия,идут,Титульный список объектов улично-дорожной сети...,Unknown,2643801297,"{coordinates=[[[37.673245668, 55.738699528], [...","{coordinates=[37.67313838, 55.738919987], type..."


In [ ]:
df['WorkYear'] = df['WorkYear'].astype(np.int32)

In [ ]:
df['WorkReason'].value_counts()

,count
WorkReason,
"Титульный список объектов улично-дорожной сети города Москвы, подлежащих ремонту",5015


In [ ]:
df['OnTerritoryOfMoscow'].value_counts()

,count
OnTerritoryOfMoscow,
да,3567
нет,23


Посмотрим некоторые категориальные признаки

In [ ]:
df['AdmArea'].value_counts()

,count
AdmArea,
Центральный административный округ,591
Северо-Восточный административный округ,457
Северный административный округ,424
Восточный административный округ,348
Западный административный округ,343
Южный административный округ,304
Северо-Западный административный округ,282
Юго-Восточный административный округ,250
Юго-Западный административный округ,204


Поскольку данные необходимы для анализа, в т.ч. плотности ремонтных работ по районам и округам, чтобы данные не были утеряны, проведем explode поля, чтобы в поле хранить только одно наименование

In [ ]:
def convert_to_list(value):
    # Удаляем квадратные скобки и разделяем по запятой
    items = value.strip('[]').split(', ')
    # Возвращаем список
    return items if items != [''] else []

In [ ]:
df['AdmArea'] = df['AdmArea'].apply(convert_to_list)

In [ ]:
df['District'] = df['District'].apply(convert_to_list)

In [ ]:
df = df.explode('AdmArea')

In [ ]:
df = df.explode('District')

Преобразуем набор geoData

In [ ]:
def extract_coordinates(geometry_str):
    pattern = r"coordinates=\[\[\[(.*?)\]\]"
    match = re.search(pattern, geometry_str)

    if match:
        coordinates_str = match.group(1)
        return f'[[{coordinates_str}]]'
    return None

df['geoData'] = df['geoData'].apply(extract_coordinates)

In [ ]:
df.head()

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,global_id,geoData,geodata_center
1,2-й Звенигородский переулок,2020,да,Центральный административный округ,Пресненский район,2020-08-14,2020-10-08,2020-08-14,2020-10-08,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1031138031,"[[37.555009201, 55.764145825], [37.55523361, 5...","{coordinates=[37.555888069, 55.765439715], typ..."
2,2-я Звенигородская улица,2020,да,Центральный административный округ,Пресненский район,2020-08-28,2020-09-13,2020-08-28,2020-09-13,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1031138034,"[[37.555076618, 55.75643665], [37.555360933, 5...","{coordinates=[37.554847163, 55.759759371], typ..."
3,2-я Черногрязская улица,2020,да,Центральный административный округ,Пресненский район,2020-08-14,2020-10-02,2020-08-14,2020-10-02,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1031138035,"[[37.547976298, 55.76482853], [37.548201604, 5...","{coordinates=[37.547257466, 55.762548058], typ..."
4,Бахрушина ул.,2020,да,Центральный административный округ,район Замоскворечье,2020-03-21,2020-06-03,2020-03-21,2020-06-03,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1031138040,"[[37.637720128, 55.731480493], [37.637988349, ...","{coordinates=[37.635806609, 55.734396363], typ..."
5,Библиотечная ул. (от ул. Б.Андроньевская до ул...,2020,да,Центральный административный округ,Таганский район,2020-03-08,2020-03-20,2020-03-08,2020-03-20,текущий ремонт асфальтобетонного покрытия,закончены,Титульный список объектов улично-дорожной сети...,ГБУ АВТОМОБИЛЬНЫЕ ДОРОГИ,1031138042,"[[37.668564862, 55.744265971], [37.668583637, ...","{coordinates=[37.66860947, 55.744302349], type..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5015 entries, 1 to 3590
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   WorksPlace           5015 non-null   object        
 1   WorkYear             5015 non-null   int32         
 2   OnTerritoryOfMoscow  5015 non-null   object        
 3   AdmArea              5015 non-null   object        
 4   District             5015 non-null   object        
 5   WorksBeginDate       5015 non-null   datetime64[ns]
 6   PlannedEndDate       5015 non-null   datetime64[ns]
 7   ActualBeginDate      4910 non-null   datetime64[ns]
 8   ActualEndDate        4689 non-null   datetime64[ns]
 9   WorksType            5015 non-null   object        
 10  WorksStatus          5015 non-null   object        
 11  WorkReason           5015 non-null   object        
 12  Customer             5015 non-null   object        
 13  global_id            5015 non-null   o

## **Сохранение обработанного датасета**

In [ ]:
df.to_csv('Moscow_alter_processed_data.csv', sep='^', index=False, encoding='utf-8')

## **Карта**

In [ ]:
# m = folium.Map(location=[55.7558, 37.6173], zoom_start=11)

# # Словарь для хранения слоев по административным округам и районам
# layer_groups = {}

# # Проход по всем строкам DataFrame
# for idx, row in df.iterrows():
#     coords = row['Geometry_line']
#     adm_area = row['AdmArea']
#     district = row['District']

#     if pd.notna(coords):
#         # Преобразование строки координат в список точек
#         points = [
#             [float(lat), float(lon)]
#             for lon, lat in (point.split(',') for point in coords.split(';'))
#         ]

#         # Создание слоя для административного округа, если его еще нет
#         if adm_area not in layer_groups:
#             layer_groups[adm_area] = {}

#         # Создание слоя для района, если его еще нет
#         if district not in layer_groups[adm_area]:
#             layer_groups[adm_area][district] = folium.FeatureGroup(name=district)

#         # Добавление линии в слой района
#         folium.PolyLine(
#             locations=points,
#             color='blue',
#             weight=3,
#             popup=f"{row['Address']} ({row['Event_type']})"
#         ).add_to(layer_groups[adm_area][district])

# # Добавление всех слоев на карту
# for adm_area, districts in layer_groups.items():
#     # Создаем общий слой для административного округа
#     adm_area_group = folium.FeatureGroup(name=adm_area)

#     for district, layer in districts.items():
#         adm_area_group.add_child(layer)  # Добавляем слой района в общий слой округа

#     adm_area_group.add_to(m)  # Добавляем общий слой округа на карту

# # Добавление переключателя слоев
# folium.LayerControl().add_to(m)

# # Сохранение карты
# m.save('map_with_districts.html')